# Classify Hymenoptera species

In [1]:
##############################################################################
#                                                                            #
#                ▄▄████████████▄ ▐                                           #
#             ▄███▀▀▀▀▀▀▀████████▐                                           #
#         ▄▀▀▀▄▄▄▄███████▄▄▄▄▀▀▀█▐                                           #
#      ▄▛ ▟█████████████████████▄▐                                           #
#    ▄█▛ ▟█         ▀▚   █████   ▐                                           #
#   ███ ██▌   ▄▄▄▖   ▐  ▐████▌   ▐    ████████ ████████  ▟████▙   ██     ██  #
#  ████ ██▌   ███▛   ▐  ▐████▌   ▐       ██    ██       ▟█▛  ▜█▙  ██     ██  #
#  ████ ▀█           ▞  █████    ▐       ██    ███████  ██        █████████  #
#  █████ █   ▄▄▄▄▄▄▄▞   ▜██▛    ▐▐       ██    ██       ▜█▙  ▟█▛  ██     ██  #
#   █████    ██████            ▐█▐       ██    ████████  ▜████▛   ██     ██  #
#    █████  ███████▙          ▟██▐                                           #
#     ▀████▙▀▀▀█████████████▀▀▀▄▄▐                                           #
#       ▀██████▄▄▄▄▀▀▀▀▀▄▄▄▄█████▐                                           #
#          ▀███████████████████▀ ▐                                           #
#             ▀▀███████████▀▀    ▐                                           #
#                                                                            #
##############################################################################

In [46]:
# import the necessary packages
import numpy as np
import cv2
import os
import torch
import torchvision
import yolo
from utils.callbacks import Callbacks

# Constants

In [67]:
DATA_PATH = "~/Projets/Hymenoptera-Recognizer/dataset/Hymenoptera.v2i.yolov5pytorch"
MODEL_PATH = "./models"

# Load dataset

In [4]:
# load pytorch dataset folder
def load_dataset(data_path):
    dataset = torchvision.datasets.ImageFolder(root=data_path,
                                               transform=torchvision.transforms.ToTensor())
    return dataset

In [5]:
dataset = load_dataset(DATA_PATH)
dataset

Dataset ImageFolder
    Number of datapoints: 585
    Root location: ./dataset/Hymenoptera.v2i.yolov5pytorch
    StandardTransform
Transform: ToTensor()

# Load model

Telecharger le modèle yolov5m.pt et le mettre dans le répertoire ".models"

In [24]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5m')

Using cache found in /home/sylvain/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-2-9 torch 1.10.2+cu102 CPU

Fusing layers... 
Model Summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


# Retrain the model with our data

```bash
git clone git@github.com:ultralytics/yolov5.git
```

In [70]:
!python yolov5/train.py --img 640 --batch 16 --epochs 5 --data {DATA_PATH}/data.yaml --weights yolov5m.pt

/home/sylvain/.pyenv/versions/3.8.9/envs/UseCase/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
train: weights=yolov5s.pt, cfg=, data=/home/sylvain/Projets/Hymenoptera-Recognizer/dataset/Hymenoptera.v2i.yolov5pytorch/data.yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=5, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: 

# Testing with a new image

In [ ]:
# load the image from disk
image = cv2.imread('./image/image_1.jpeg')
# create blob from image
blob = cv2.dnn.blobFromImage(image=image, scalefactor=0.01, size=(224, 224), mean=(104, 117, 123))

In [ ]:
# set the input blob for the neural network
net.setInput(blob)
# forward pass image blog through the model
outputs = net.forward()


In [ ]:
outputs

In [ ]:
final_outputs = outputs[0]
# make all the outputs 1D
final_outputs = final_outputs.reshape(25)
# get the class label
label_id = np.argmax(final_outputs)
# convert the output scores to softmax probabilities
probs = np.exp(final_outputs) / np.sum(np.exp(final_outputs))
# get the final highest probability
final_prob = np.max(probs) * 100.
# map the max confidence to the class label names
out_name = labels[label_id]
out_text = f"{out_name}, {final_prob:.3f}"
print(out_text)

## Metrics

In [ ]:
import sklearn.metrics as metrics

In [ ]:
# y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
# y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
metrics.confusion_matrix(y_true, y_pred, labels=labels)